objc[87510]: Class CaptureDelegate is implemented in both /Users/wu/anaconda3/envs/mediapipe/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x10ae08860) and /Users/wu/anaconda3/envs/mediapipe/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x1665e2840). One of the two will be used. Which one is undefined.
objc[87510]: Class CVWindow is implemented in both /Users/wu/anaconda3/envs/mediapipe/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x105554a68) and /Users/wu/anaconda3/envs/mediapipe/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x1665e2890). One of the two will be used. Which one is undefined.
objc[87510]: Class CVView is implemented in both /Users/wu/anaconda3/envs/mediapipe/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x105554a90) and /Users/wu/anaconda3/envs/mediapipe/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x1665e28b8). One of the two will be used. Which one is undefined.
objc[87510]

In [2]:
import time
import random
from selenium import webdriver

# Initialize the Chrome WebDriver
driver = webdriver.Chrome()

# Navigate to a web page
driver.get("https://www.example.com")

def scale_up():
    # Create a JavaScript code snippet to change the zoom level
    js_code = f"document.body.style.transform = 'scale(2)';"

    # Execute the JavaScript code to scale the web page
    driver.execute_script(js_code)

def scale_down():
    # Create a JavaScript code snippet to change the zoom level
    js_code = f"document.body.style.transform = 'scale(0.5)';"

    # Execute the JavaScript code to scale the web page
    driver.execute_script(js_code)

In [3]:
import cv2
import mediapipe as mp
import time
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# STEP 2: Create an GestureRecognizer object.
base_options = python.BaseOptions(model_asset_path='gesture_recognizer.task')
options = vision.GestureRecognizerOptions(base_options=base_options)
recognizer = vision.GestureRecognizer.create_from_options(options)

# Initialize MediaPipe Hand components
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# Define your custom gesture recognition function
def recognize_gesture(hand_landmarks):
    """
    recog_result.gestures[0][0].category_name:
        Open palm (Open_Palm)
        Pointing up (Pointing_Up)
        Thumbs down (Thumb_Down)
        Thumbs up (Thumb_Up)
        Victory (Victory)
        Love (ILoveYou)
    recog_result.gestures[0][0].score:
        0.0 ~ 1.0
    """
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)
    recognition_result = recognizer.recognize(mp_image)
    return recognition_result

# Start capturing video from the webcam
cap = cv2.VideoCapture(0)

# Initialize MediaPipe Hands with static_image_mode=False for real-time detection
with mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to RGB for MediaPipe processing
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame and get hand landmarks
        result = hands.process(frame_rgb)

        # Draw hand landmarks on the frame
        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Call your custom gesture recognition function
                recog_result = recognize_gesture(hand_landmarks)
                if recog_result.gestures:
                    gesture = recog_result.gestures[0][0].category_name
                    score = recog_result.gestures[0][0].score
                    if gesture != 'None':
                        print(recog_result.gestures, score)
                    if gesture == 'Thumb_Up' and score > 0.8:
                        scale_up()
        # Display the frame
        cv2.imshow('Gesture Recognition', frame)

        # Break the loop when the 'q' key is pressed
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()



KeyboardInterrupt: 

In [ ]:
recog_result.gestures[0][0].category_name

IndexError: list index out of range

In [ ]:
type(recog_result)

mediapipe.tasks.python.vision.gesture_recognizer.GestureRecognizerResult